# Titanic - Predictive Modeling!
In our last session, we focused on performing feature engineering on the raw Titanic dataset. At the end of that session, we exported our cleaned dataset into another CSV file. We are now ready to start our predictive modeling process!

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import cloudpickle
import pandas as pd
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Loading in the cleaned datasets
X = pd.read_csv('../data/clean/X.csv')
y = pd.read_csv('../data/clean/y.csv')

In [6]:
# Viewing the first few rows of the X dataset
X.head()

,Pclass,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan,Age_child,Age_teen,Age_young_adult,Age_adult,Age_elder
0,3,1,0,7.2500,1,0,1,0,0,0,0,0,1,0,0
1,1,1,0,71.2833,0,1,0,1,0,0,0,0,0,1,0
2,3,0,0,7.9250,0,1,1,0,0,0,0,0,1,0,0
3,1,1,0,53.1000,0,1,1,0,0,0,0,0,0,1,0
4,3,0,0,8.0500,1,0,1,0,0,0,0,0,0,1,0


In [7]:
# Viewing the first few rows of the y dataset
y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


## Data Separation
When working with a training dataset, it is a good idea to hold out a portion of the data so that we have something we can validate the model against. In the cell below, we will use Scikit-Learn's `train_test_split` functionality to split the data into respective training and validation sets.

In [8]:
# Splitting the datasets between training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
# Viewing the first few rows of the X_train dataset
X_train.head()

,Pclass,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,Embarked_nan,Age_child,Age_teen,Age_young_adult,Age_adult,Age_elder
331,1,0,0,28.5000,1,0,1,0,0,0,0,0,0,1,0
733,2,0,0,13.0000,1,0,1,0,0,0,0,0,1,0,0
382,3,0,0,7.9250,1,0,1,0,0,0,0,0,0,1,0
704,3,1,0,7.8542,1,0,1,0,0,0,0,0,1,0,0
813,3,4,2,31.2750,0,1,1,0,0,0,1,0,0,0,0


In [10]:
# Viewing the first few rows of the y_train dataset
y_train.head()

,Survived
331,0
733,0
382,0
704,0
813,0


In [11]:
# Checking to see that the 20% split worked properly
len(X_val) / len(X)

0.20089786756453423

## Hyperparameter Tuning

In [12]:
# Instantiating a Random Forest Classifier object
rfc_gridsearch = RandomForestClassifier()

In [13]:
# Defining the parameter grid for hyperparameter tuning
params = {'n_estimators': [10, 50, 100],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 5],
          'max_depth': [10, 20, 50]
         }

In [14]:
# Instantiating the GridSearchCV object
hyperparameter_tuner = GridSearchCV(estimator = rfc_gridsearch,
                                    param_grid = params)

In [15]:
# Running the hyperparameter tuning job
hyperparameter_tuner.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 20, 50],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100]})

In [16]:
# Viewing the best parameters from the hyperparameter tuning job
hyperparameter_tuner.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 50}

## Model Training

In [17]:
# Instantiating a new Random Forest Classifier object
rfc_model = RandomForestClassifier(n_estimators = 50,
                                   max_depth = 20,
                                   min_samples_split = 10,
                                   min_samples_leaf = 2)

In [18]:
# Performing the model training
rfc_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=50)

## Model Validation

In [19]:
# Getting predictions on the X_val dataset using the trained RFC model
val_preds = rfc_model.predict(X_val)

In [20]:
# Getting the metrics with the validation dataset
val_accuracy = accuracy_score(y_val, val_preds)
val_roc_auc = roc_auc_score(y_val, val_preds)
val_confusion_matrix = confusion_matrix(y_val, val_preds)

In [21]:
# Printing out the validation metrics
print(f'Accuracy Score: {val_accuracy}')
print(f'ROC-AUC Score: {val_roc_auc}')
print(f'Confusion Matrix: \n{val_confusion_matrix}')

Accuracy Score: 0.8156424581005587
ROC-AUC Score: 0.798970398970399
Confusion Matrix: 
[[94 11]
 [22 52]]


## Saving out a Simple Model

In [22]:
# Saving the RFC model to a pickle
with open('../models/rfc_model.pkl', 'wb') as f:
    cloudpickle.dump(rfc_model, f)

## Loading our Trained Model

In [23]:
# Loading in the RFC model from serialized file
with open('../models/rfc_model.pkl', 'rb') as f:
    rfc_loaded_model = cloudpickle.load(f)

In [24]:
# Getting predictions with the loaded model
loaded_preds = rfc_loaded_model.predict(X_val)

In [25]:
# Showing the metrics with the loaded preds
val_accuracy = accuracy_score(y_val, loaded_preds)
val_roc_auc = roc_auc_score(y_val, loaded_preds)
val_confusion_matrix = confusion_matrix(y_val, loaded_preds)

print(f'Accuracy Score: {val_accuracy}')
print(f'ROC-AUC Score: {val_roc_auc}')
print(f'Confusion Matrix: \n{val_confusion_matrix}')

Accuracy Score: 0.8156424581005587
ROC-AUC Score: 0.798970398970399
Confusion Matrix: 
[[94 11]
 [22 52]]


## Creating a Full Pipeline
In our livestream on Sept. 9, we focused on training a model and just saving that model alone. It is possible to save a serialized pickle file that not only contains the ability to perform inference on cleaned data but can also do that data cleansing itself. In this section, we'll be taking everything we've done so far to create a full ML inference pipeline.

### Loading Raw Data
Since our pipeline is going to take data in pretty much it's purest, rawest form, we are going to load in that original raw dataset we downloaded from Kaggle instead of working with our already cleaned data.

In [50]:
# Loading in the raw Titanic training data
df_raw = pd.read_csv('../data/raw/train.csv')

In [25]:
# Separating predictor value from the remainder of the dataset
X = df_raw.drop(columns = ['Survived'])
y = df_raw[['Survived']]

In [52]:
# Reloading the raw data to ensure we have the original columns including Age
df_raw = pd.read_csv('../data/raw/train.csv')
X = df_raw.drop(columns = ['Survived'])
y = df_raw[['Survived']]

# Performing training / validation dataset split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42)

In [53]:
print("Columns in X_train:")
print(X_train.columns.tolist())
print("\nFirst few rows of X_train with Age column:")
X_train.head()

Columns in X_train:
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

First few rows of X_train with Age column:


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
298,299,1,"Saalfeld, Mr. Adolphe",male,NaN,0,0,19988,30.5000,C106,S
884,885,3,"Sutehall, Mr. Henry Jr",male,25.00,0,0,SOTON/OQ 392076,7.0500,NaN,S
247,248,2,"Hamalainen, Mrs. William (Anna)",female,24.00,0,2,250649,14.5000,NaN,S
478,479,3,"Karlsson, Mr. Nils August",male,22.00,0,0,350060,7.5208,NaN,S
305,306,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S


### Helper Functions
As part of our feature engineering, you might recall that we did some special things to create certain features. Before we jump into creating our pipeline, we'll need to package that feature engineering as their own respective Python functions. You'll notice this code is pretty much copy/pasted as is from our feature engineering notebook.

In [17]:
# Creating a function to appropriately engineer the 'Age' column
def create_age_bins(df):
    '''Engineers age bin variables for the pipeline'''
    
    # Filling any null values with the median age of 28.0
    median_age = 28.0
    df['Age'].fillna(median_age, inplace = True)
    
    # Establishing our bins values and names
    bin_labels = ['child', 'teen', 'young_adult', 'adult', 'elder']
    bin_values = [-1, 12, 19, 30, 60, 100]
    
    # Applying "Age" binning with Pandas cut
    age_bins = pd.cut(df['Age'], bins = bin_values, labels = bin_labels)
    df_age_bins = pd.DataFrame(age_bins)
    
    # Dropping the original "Age" column
    df.drop(columns = ['Age'], inplace = True)
    
    # Concatenating the new "Age" column to the original DataFrame
    df = pd.concat([df, df_age_bins], axis = 1)
    
    return df

In [29]:
# Test the age binning function with a copy to avoid modifying original data
X_train_copy = X_train.copy()
a = create_age_bins(X_train_copy)
print("Columns after age binning:")
print(a.columns.tolist())
a.head()

Columns after age binning:
['PassengerId', 'Pclass', 'Name', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age']


,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age
298,299,1,"Saalfeld, Mr. Adolphe",male,0,0,19988,30.5000,C106,S,young_adult
884,885,3,"Sutehall, Mr. Henry Jr",male,0,0,SOTON/OQ 392076,7.0500,NaN,S,young_adult
247,248,2,"Hamalainen, Mrs. William (Anna)",female,0,2,250649,14.5000,NaN,S,young_adult
478,479,3,"Karlsson, Mr. Nils August",male,0,0,350060,7.5208,NaN,S,young_adult
305,306,1,"Allison, Master. Hudson Trevor",male,1,2,113781,151.5500,C22 C26,S,child


### Pipeline Creation
Now that we have created our helper functions to perform the feature engineering, we are ready to begin packaging everything as a single, unified pipeline.

In [34]:
# Creating the "Age" binning function transformer as the first step into our modeling pipeline
age_binner = FunctionTransformer(create_age_bins, validate = False)

In [19]:
age_binner

FunctionTransformer(func=<function create_age_bins at 0x0000021F7FE35DC0>)

In [35]:
# Creating the data preprocessor that will perform our feature engineering
data_preprocessor = ColumnTransformer(transformers = [
    ('ohe_engineering', OneHotEncoder(use_cat_names = True, handle_unknown = 'ignore'), ['Age', 'Sex', 'Embarked']),
    ('columns_to_drop', 'drop', ['PassengerId', 'Name', 'Ticket', 'Cabin'])],
                                      remainder = 'passthrough'
)

In [37]:
# Creating the full inference pipeline
rfc_pipeline = Pipeline(steps = [
    ('age_binning', age_binner),
    ('feature_engineering', data_preprocessor),
    ('predictive_modeling', RandomForestClassifier(n_estimators = 50,
                                                   max_depth = 20,
                                                   min_samples_split = 10,
                                                   min_samples_leaf = 2))
])

In [49]:
X_train

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
298,299,1,"Saalfeld, Mr. Adolphe",male,0,0,19988,30.5000,C106,S
884,885,3,"Sutehall, Mr. Henry Jr",male,0,0,SOTON/OQ 392076,7.0500,NaN,S
247,248,2,"Hamalainen, Mrs. William (Anna)",female,0,2,250649,14.5000,NaN,S
478,479,3,"Karlsson, Mr. Nils August",male,0,0,350060,7.5208,NaN,S
305,306,1,"Allison, Master. Hudson Trevor",male,1,2,113781,151.5500,C22 C26,S
...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,0,0,343120,7.6500,NaN,S
270,271,1,"Cairns, Mr. Alexander",male,0,0,113798,31.0000,NaN,S
860,861,3,"Hansen, Mr. Claus Peter",male,2,0,350026,14.1083,NaN,S
435,436,1,"Carter, Miss. Lucile Polk",female,1,2,113760,120.0000,B96 B98,S


In [48]:
# Training the inference pipeline with the training data
rfc_pipeline.fit(X_train, y_train)

KeyError: 'Age'

In [10]:
# Generating inferences on the validation dataset with the trained RFC pipeline
val_preds = pd.DataFrame(rfc_pipeline.predict(X_val))

In [11]:
# Getting the metrics with the validation dataset
val_accuracy = accuracy_score(y_val, val_preds)
val_roc_auc = roc_auc_score(y_val, val_preds)
val_confusion_matrix = confusion_matrix(y_val, val_preds)

In [12]:
# Printing out the validation metrics
print(f'Accuracy Score: {val_accuracy}')
print(f'ROC-AUC Score: {val_roc_auc}')
print(f'Confusion Matrix: \n{val_confusion_matrix}')

Accuracy Score: 0.8340807174887892
ROC-AUC Score: 0.8185477108837833
Confusion Matrix: 
[[120  14]
 [ 23  66]]


In [13]:
# Saving the pipeline to a serialized pickle file
with open('../models/rfc_pipeline.pkl', 'wb') as f:
    cloudpickle.dump(rfc_pipeline, f)

### Loading Our Trained Pipeline

In [ ]:
# Loading in the trained RFC pipeline from the serialized pickle file
with open('../models/rfc_pipeline.pkl', 'rb') as f:
    loaded_rfc_pipeline = cloudpickle.load(f)

In [ ]:
# Getting inferences with the loaded RFC pipeline
val_preds = pd.DataFrame(loaded_rfc_pipeline.predict(X_val))

In [46]:
# Getting the metrics with the validation dataset
val_accuracy = accuracy_score(y_val, val_preds)
val_roc_auc = roc_auc_score(y_val, val_preds)
val_confusion_matrix = confusion_matrix(y_val, val_preds)

# Printing out the validation metrics
print(f'Accuracy Score: {val_accuracy}')
print(f'ROC-AUC Score: {val_roc_auc}')
print(f'Confusion Matrix: \n{val_confusion_matrix}')

Accuracy Score: 0.8340807174887892
ROC-AUC Score: 0.8166610766392757
Confusion Matrix: 
[[121  13]
 [ 24  65]]
